In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import re
import string

import spacy

import gensim
from gensim import corpora
from nltk.stem import PorterStemmer

# libraries for visualization
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


from nltk.corpus import stopwords

en = spacy.load('en_core_web_sm')
stpwrds = en.Defaults.stop_words
all_stopwords = stpwrds.union(stopwords.words('english'))

nlp = spacy.load('en_core_web_sm', disable=['parser','ner'])

pd.set_option('display.max_colwidth', -1)

/opt/homebrew/Cellar/jupyterlab/3.3.2/libexec/lib/python3.9/site-packages/seaborn/rcmod.py:82: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
/opt/homebrew/Cellar/jupyterlab/3.3.2/libexec/lib/python3.9/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


In [2]:
review_data= pd.read_csv("Reviews.csv")
print(review_data.head(2))
print(len(review_data))
print('Unique Products')
print(len(review_data.groupby('ProductId')))
print('Unique Users')
print(len(review_data.groupby('UserId')))

   Id   ProductId          UserId ProfileName  HelpfulnessNumerator  \
0  1   B001E4KFG0  A3SGXH7AUHU8GW  delmartian  1                      
1  2   B00813GRG4  A1D87F6ZCVE5NK  dll pa      0                      

   HelpfulnessDenominator  Score        Time                Summary  \
0  1                       5      1303862400  Good Quality Dog Food   
1  0                       1      1346976000  Not as Advertised       

                                                                                                                                                                                                                                                                      Text  
0  I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.  
1  Product arrived labeled as Jumb

In [3]:
review_data.Text

0         I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.                                                                                                                                                                                                                                                      
1         Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".                                                                                                                                                                                                                                                                                        

In [4]:
def clean_text(text): 
    delete_dict = {sp_character: '' for sp_character in string.punctuation}
    delete_dict[' '] = ' ' 
    table = str.maketrans(delete_dict)
    text1 = text.translate(table)
    textArr= text1.split()
    text2 = ' '.join([w for w in textArr if ( not w.isdigit() and  ( not w.isdigit() and len(w)>3))]) 
    
    return text2.lower()

In [5]:
# delete_dict = {sp_character: '' for sp_character in string.punctuation}
# delete_dict[' '] = ' '
# delete_dict
# table = str.maketrans(delete_dict)
# print(table)
# text1 = review_data['Text'].tolist()[0].translate(table)
# text1
# textArr = text1.split()
# print(textArr)
# text2 = ' '.join([w for w in textArr if ( not w.isdigit() and  ( not w.isdigit() and len(w)>3))])
# text2

In [6]:
review_data.dropna(axis = 0, how ='any',inplace=True) 


review_data['Text'] = review_data['Text'].apply(clean_text)
review_data['Num_words_text'] = review_data['Text'].apply(lambda x:len(str(x).split())) 

print('-------Dataset --------')
print(review_data['Score'].value_counts())
print(len(review_data))
print('-------------------------')
max_review_data_sentence_length = review_data['Num_words_text'].max()

mask = (review_data['Num_words_text'] < 100) & (review_data['Num_words_text'] >=20)
df_short_reviews = review_data[mask]
df_sampled = df_short_reviews.groupby('Score').apply(lambda x: x.sample(n=20000)).reset_index(drop = True)

print('No of Short reviews')
print(len(df_short_reviews))

-------Dataset --------
5    363111
4    80655 
1    52264 
3    42638 
2    29743 
Name: Score, dtype: int64
568411
-------------------------
No of Short reviews
373281


In [7]:
df_sampled.Text

0        really disappointed with this because expected mroe from loose leaf wife thinks smells like marijuana couldnt taste even hint mint total waste money                                                                                                                                                                                                                                                                                                                                                                                                                                     
1        always looking quick healthy easy make snacks work love microwave popcorn various cheeses therefore easily imagine excitement upon discovering orville reddenbachers flavor cheddar cheese popcorn grocery store after trying three bags over past week have never been sickeningly disappointed thought maybe first stale tried with same results doesnt even taste like popcorn alone cheese tried light sprinkle salt tha

In [8]:
sample_list = df_sampled.Text.tolist()
for_end = df_sampled.Text.tolist()
sample_list

['really disappointed with this because expected mroe from loose leaf wife thinks smells like marijuana couldnt taste even hint mint total waste money',
 'always looking quick healthy easy make snacks work love microwave popcorn various cheeses therefore easily imagine excitement upon discovering orville reddenbachers flavor cheddar cheese popcorn grocery store after trying three bags over past week have never been sickeningly disappointed thought maybe first stale tried with same results doesnt even taste like popcorn alone cheese tried light sprinkle salt that made worse tasted nastybr from only buying generic popcorn from kroger marsh aldis adding seasoning',
 'there many artificial ingredients this made read smell bourbon which just above salt then have better nose than hound want sodium benzoate yellow yellow blue partially hydrogenated soybean diglycerides artifical flavor sodium propionate this your fruitcake wish could send mine flavor very disappointing what expect with those 

In [9]:
warnings.filterwarnings('ignore')

def remove_stopwords(text):
    finalList = []
    for i in text:
        textArr = i.split(' ')    
        rem_text = ' '.join([i for i in textArr if i not in all_stopwords])
        finalList.append(rem_text)
    return finalList


# remove stopwords from the text
sample_list = remove_stopwords(sample_list)
sample_list

['disappointed expected mroe loose leaf wife thinks smells like marijuana couldnt taste hint mint total waste money',
 'looking quick healthy easy snacks work love microwave popcorn cheeses easily imagine excitement discovering orville reddenbachers flavor cheddar cheese popcorn grocery store trying bags past week sickeningly disappointed thought maybe stale tried results doesnt taste like popcorn cheese tried light sprinkle salt worse tasted nastybr buying generic popcorn kroger marsh aldis adding seasoning',
 'artificial ingredients read smell bourbon salt better nose hound want sodium benzoate yellow yellow blue partially hydrogenated soybean diglycerides artifical flavor sodium propionate fruitcake wish send flavor disappointing expect chemical need nuns cake think monestary ploy',
 'ordered sampler pack flavored coffee grove square cappuccino mixes drink flavored coffee dont partially hydrogenated vegetable contain coconut soybean sunflower palm andor canola sugar corn syrup solid

In [10]:
"""Returns lemmatized text"""

def lemmatize(text):
    #text_list = text.tolist()
    splem = []
    wordlist = []
    for i in range(100):
        wordlist.append(text[i])
        lst = word_tokenize(text[i])
        #print(lst)
        #wordlist.append(lst)
        doc = nlp(' '.join(map(str,lst)))
        lem = " ".join([token.lemma_ for token in doc])
        #print(lem)
        splem.append(lem)
    #return(' '.join(map(str,splem)))
    return splem

In [11]:
# """Returns lemmatized text"""

# nlp = spacy.load('en_core_web_sm', disable=['parser','ner'])
# def lemmatize(text):
#     #text_list = text.tolist()
#     splem = []
#     wordlist = []
#     for i in text:
#         wordlist.append(i)
#         lst = word_tokenize(i)
#         #print(lst)
#         #wordlist.append(lst)
#         doc = nlp(' '.join(map(str,lst)))
#         lem = " ".join([token.lemma_ for token in doc])
#         #print(lem)
#         splem.append(lem)
#     #return(' '.join(map(str,splem)))
#     return splem

In [12]:
lemmatized = lemmatize(sample_list)
lemmatized

['disappointed expected mroe loose leaf wife think smell like marijuana could not taste hint mint total waste money',
 'look quick healthy easy snack work love microwave popcorn cheese easily imagine excitement discover orville reddenbacher flavor cheddar cheese popcorn grocery store try bag past week sickeningly disappointed thought maybe stale try result do not taste like popcorn cheese try light sprinkle salt worse taste nastybr buy generic popcorn kroger marsh aldi add seasoning',
 'artificial ingredient read smell bourbon salt well nose hound want sodium benzoate yellow yellow blue partially hydrogenate soybean diglyceride artifical flavor sodium propionate fruitcake wish send flavor disappointing expect chemical need nun cake think monestary ploy',
 'order sampler pack flavor coffee grove square cappuccino mix drink flavor coffee do not partially hydrogenate vegetable contain coconut soybean sunflower palm andor canola sugar corn syrup solid instant coffee sodium caseinate salt n

In [13]:
tokenized_reviews = remove_stopwords(lemmatized)
#tokenized_reviews = list(tokenized_reviews.split(' '))
tokenized_reviews

['disappointed expected mroe loose leaf wife think smell like marijuana taste hint mint total waste money',
 'look quick healthy easy snack work love microwave popcorn cheese easily imagine excitement discover orville reddenbacher flavor cheddar cheese popcorn grocery store try bag past week sickeningly disappointed thought maybe stale try result taste like popcorn cheese try light sprinkle salt worse taste nastybr buy generic popcorn kroger marsh aldi add seasoning',
 'artificial ingredient read smell bourbon salt nose hound want sodium benzoate yellow yellow blue partially hydrogenate soybean diglyceride artifical flavor sodium propionate fruitcake wish send flavor disappointing expect chemical need nun cake think monestary ploy',
 'order sampler pack flavor coffee grove square cappuccino mix drink flavor coffee partially hydrogenate vegetable contain coconut soybean sunflower palm andor canola sugar corn syrup solid instant coffee sodium caseinate salt natural artificial flavor coco

In [14]:
tokenized_reviews_check = [d.split() for d in tokenized_reviews]
tokenized_reviews_check 

[['disappointed',
  'expected',
  'mroe',
  'loose',
  'leaf',
  'wife',
  'think',
  'smell',
  'like',
  'marijuana',
  'taste',
  'hint',
  'mint',
  'total',
  'waste',
  'money'],
 ['look',
  'quick',
  'healthy',
  'easy',
  'snack',
  'work',
  'love',
  'microwave',
  'popcorn',
  'cheese',
  'easily',
  'imagine',
  'excitement',
  'discover',
  'orville',
  'reddenbacher',
  'flavor',
  'cheddar',
  'cheese',
  'popcorn',
  'grocery',
  'store',
  'try',
  'bag',
  'past',
  'week',
  'sickeningly',
  'disappointed',
  'thought',
  'maybe',
  'stale',
  'try',
  'result',
  'taste',
  'like',
  'popcorn',
  'cheese',
  'try',
  'light',
  'sprinkle',
  'salt',
  'worse',
  'taste',
  'nastybr',
  'buy',
  'generic',
  'popcorn',
  'kroger',
  'marsh',
  'aldi',
  'add',
  'seasoning'],
 ['artificial',
  'ingredient',
  'read',
  'smell',
  'bourbon',
  'salt',
  'nose',
  'hound',
  'want',
  'sodium',
  'benzoate',
  'yellow',
  'yellow',
  'blue',
  'partially',
  'hydrogen

In [15]:
tokenized_reviews  = [d.split() for d in tokenized_reviews ]
dictionary = corpora.Dictionary(tokenized_reviews)
print(dictionary)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in tokenized_reviews]

Dictionary(1322 unique tokens: ['disappointed', 'expected', 'hint', 'leaf', 'like']...)


In [16]:
print(doc_term_matrix)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)], [(0, 1), (4, 1), (11, 2), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 3), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 4), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 3), (54, 1), (55, 1), (56, 1)], [(10, 1), (12, 1), (26, 2), (45, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 2), (81, 1), (82, 1), (83, 1), (84, 2)], [(26, 3), (45, 1), (58, 2), (64, 1), (69, 1), (75, 1), (80, 1), (81, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 2), (90, 1), (91, 1), (92, 1), (93, 1), (94, 3), (95, 1), (96, 

In [17]:
# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=10, random_state=100,
                chunksize=1000, passes=50,iterations=100)

In [18]:
lda_model.print_topics()

[(0,
  '0.023*"ball" + 0.019*"container" + 0.014*"think" + 0.014*"treat" + 0.014*"chew" + 0.009*"try" + 0.009*"product" + 0.009*"away" + 0.009*"good" + 0.009*"liquid"'),
 (1,
  '0.020*"like" + 0.012*"flavor" + 0.008*"package" + 0.008*"think" + 0.008*"sugar" + 0.008*"free" + 0.008*"true" + 0.008*"calorie" + 0.008*"vendor" + 0.008*"chew"'),
 (2,
  '0.022*"flavor" + 0.016*"order" + 0.016*"natural" + 0.014*"coffee" + 0.014*"taste" + 0.012*"ingredient" + 0.009*"amazon" + 0.009*"think" + 0.008*"look" + 0.008*"jerky"'),
 (3,
  '0.028*"taste" + 0.022*"like" + 0.015*"chocolate" + 0.012*"smell" + 0.012*"food" + 0.009*"come" + 0.009*"price" + 0.009*"coffee" + 0.009*"tin" + 0.006*"love"'),
 (4,
  '0.028*"like" + 0.024*"flavor" + 0.013*"taste" + 0.011*"sell" + 0.009*"corn" + 0.009*"zico" + 0.009*"orange" + 0.007*"think" + 0.007*"water" + 0.007*"item"'),
 (5,
  '0.020*"food" + 0.014*"product" + 0.014*"like" + 0.011*"good" + 0.009*"sugar" + 0.009*"hard" + 0.009*"coffee" + 0.009*"consume" + 0.009*"tas

In [19]:
print (lda_model.print_topics(num_topics=10, num_words=5))

[(0, '0.023*"ball" + 0.019*"container" + 0.014*"think" + 0.014*"treat" + 0.014*"chew"'), (1, '0.020*"like" + 0.012*"flavor" + 0.008*"package" + 0.008*"think" + 0.008*"sugar"'), (2, '0.022*"flavor" + 0.016*"order" + 0.016*"natural" + 0.014*"coffee" + 0.014*"taste"'), (3, '0.028*"taste" + 0.022*"like" + 0.015*"chocolate" + 0.012*"smell" + 0.012*"food"'), (4, '0.028*"like" + 0.024*"flavor" + 0.013*"taste" + 0.011*"sell" + 0.009*"corn"'), (5, '0.020*"food" + 0.014*"product" + 0.014*"like" + 0.011*"good" + 0.009*"sugar"'), (6, '0.017*"product" + 0.013*"know" + 0.012*"taste" + 0.011*"buy" + 0.011*"candy"'), (7, '0.019*"taste" + 0.019*"flavor" + 0.014*"water" + 0.011*"drink" + 0.011*"food"'), (8, '0.022*"food" + 0.014*"like" + 0.012*"potato" + 0.012*"weight" + 0.008*"package"'), (9, '0.029*"product" + 0.013*"purchase" + 0.013*"taste" + 0.013*"slim" + 0.011*"coffee"')]


In [20]:
count = 0
for i in lda_model[doc_term_matrix]:
    print("doc : ",count,i)
    count = count + 1

doc :  0 [(2, 0.94704854)]
doc :  1 [(6, 0.98301655)]
doc :  2 [(9, 0.9749969)]
doc :  3 [(2, 0.9863616)]
doc :  4 [(2, 0.97272336)]
doc :  5 [(4, 0.9399893)]
doc :  6 [(3, 0.9590868)]
doc :  7 [(4, 0.9499894)]
doc :  8 [(4, 0.15009482), (6, 0.8328806)]
doc :  9 [(8, 0.9816287)]
doc :  10 [(7, 0.9756719)]
doc :  11 [(1, 0.9666642)]
doc :  12 [(5, 0.9608648)]
doc :  13 [(0, 0.9249892)]
doc :  14 [(2, 0.9865654)]
doc :  15 [(6, 0.9526231)]
doc :  16 [(4, 0.9735258)]
doc :  17 [(7, 0.98199755)]
doc :  18 [(5, 0.9689603)]
doc :  19 [(2, 0.98234975)]
doc :  20 [(9, 0.9437415)]
doc :  21 [(3, 0.98301667)]
doc :  22 [(6, 0.94374114)]
doc :  23 [(7, 0.9854821)]
doc :  24 [(4, 0.9774959)]
doc :  25 [(9, 0.9756732)]
doc :  26 [(8, 0.9249904)]
doc :  27 [(4, 0.94999194)]
doc :  28 [(5, 0.984998)]
doc :  29 [(7, 0.9742823)]
doc :  30 [(1, 0.9780466)]
doc :  31 [(1, 0.9590844)]
doc :  32 [(8, 0.9833315)]
doc :  33 [(7, 0.9590846)]
doc :  34 [(9, 0.94999456)]
doc :  35 [(7, 0.9653794)]
doc :  36 [(4

In [21]:
warnings.filterwarnings('ignore')
# Visualize the topics
#https://github.com/bmabey/pyLDAvis
#https://speakerdeck.com/bmabey/visualizing-topic-models
#pyLDAvis.enable_notebook()
#vis = pyLDAvis.gensim_models.prepare(lda_model, doc_term_matrix, dictionary)
#vis
pyLDAvis.enable_notebook()

# feed the LDA model into the pyLDAvis instance
lda_viz = gensimvis.prepare(lda_model,doc_term_matrix,dictionary,mds='mmds')
lda_viz

/opt/homebrew/Cellar/jupyterlab/3.3.2/libexec/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/homebrew/Cellar/jupyterlab/3.3.2/libexec/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/homebrew/Cellar/jupyterlab/3.3.2/libexec/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/homebrew/Cellar/jupyterlab/3.3.2/libexec/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.205154  0.026450  1       1        16.717932
7      0.099243  0.167732  2       1        16.557633
8     -0.142816 -0.142282  3       1        12.091699
6     -0.052032  0.195584  4       1        11.281478
4      0.136645 -0.140604  5       1        10.880070
5     -0.187253  0.079552  6       1        10.330475
9      0.003691 -0.172744  7       1        8.148041 
3     -0.121535 -0.019986  8       1        6.545652 
1      0.002235  0.037379  9       1        4.325375 
0      0.056669 -0.031081  10      1        3.121645 , topic_info=          Term       Freq      Total Category  logprob  loglift
202  product    28.000000  28.000000  Default  30.0000  30.0000
325  food       28.000000  28.000000  Default  29.0000  29.0000
4    like       38.000000  38.000000  Default  28.0000  28.0000
262  chocolate  6.000000   6.000000   Default  27.0000  27.0000
26   flavor     38.000000  38.000000  Default  26.0000  26.0000
..      ...           ...        ...      ...      ...      ...
731  calorie    0.476407   3.517970   Topic10 -5.3038   1.4674 
431  run        0.476406   2.593490   Topic10 -5.3038   1.7723 
369  stuck      0.476406   1.812240   Topic10 -5.3038   2.1308 
251  pick       0.476406   1.833542   Topic10 -5.3038   2.1191 
789  fast       0.476406   1.821440   Topic10 -5.3038   2.1257 

[603 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1006  2      0.893382  acid    
313   1      0.686719  acidbr  
165   1      0.236298  actually
165   4      0.472597  actually
165   8      0.236298  actually
...  ..           ...       ...
84    7      0.370797  yellow  
1212  5      0.725086  yoga    
601   2      0.893381  yuck    
682   2      0.232271  zico    
682   5      0.696814  zico    

[909 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 8, 9, 7, 5, 6, 10, 4, 2, 1])

In [22]:
print('\nPerplexity: ', lda_model.log_perplexity(doc_term_matrix,total_docs=10000))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_model, texts=tokenized_reviews, dictionary=dictionary , coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.102395487670675

Coherence Score:  0.37497252031553197


In [23]:
# def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
#     """
#     Compute c_v coherence for various number of topics

#     Parameters:
#     ----------
#     dictionary : Gensim dictionary
#     corpus : Gensim corpus
#     texts : List of input texts
#     limit : Max num of topics

#     Returns:
#     -------
#     model_list : List of LDA topic models
#     coherence_values : Coherence values corresponding to the LDA model with respective number of topics
#     """
#     coherence_values = []
#     model_list = []
#     for num_topics in range(start, limit, step):
#         model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)
#         model_list.append(model)
#         coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
#         coherence_values.append(coherencemodel.get_coherence())

#     return model_list, coherence_values

In [24]:
#model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=doc_term_matrix, texts=tokenized_reviews, start=2, limit=50, step=1)

In [25]:
# # Show graph
# limit=50; start=2; step=1;
# x = range(start, limit, step)
# plt.plot(x, coherence_values)
# plt.xlabel("Num Topics")
# plt.ylabel("Coherence score")
# plt.legend(("coherence_values"), loc='best')
# plt.show()# Print the coherence scores

In [26]:
# # Print the coherence scores
# for m, cv in zip(x, coherence_values):
#     print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [27]:
# optimal_model = model_list[8]
# model_topics = optimal_model.show_topics(formatted=False)
# optimal_model.print_topics(num_words=10)

In [28]:
# #import pyLDAvis
# #import pyLDAvis.gensim_models as gensimvis
# pyLDAvis.enable_notebook()

# # feed the LDA model into the pyLDAvis instance
# lda_viz = gensimvis.prepare(optimal_model,doc_term_matrix,dictionary,mds='mmds')
# lda_viz

In [29]:
def format_topics_sentences(ldamodel=lda_model, corpus=doc_term_matrix, texts=for_end):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=doc_term_matrix, texts=for_end)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(50)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,2.0,0.9470,"flavor, order, natural, coffee, taste, ingredient, amazon, think, look, jerky",really disappointed with this because expected mroe from loose leaf wife thinks smells like marijuana couldnt taste even hint mint total waste money
1,1,6.0,0.9830,"product, know, taste, buy, candy, try, store, tell, time, popcorn",always looking quick healthy easy make snacks work love microwave popcorn various cheeses therefore easily imagine excitement upon discovering orville reddenbachers flavor cheddar cheese popcorn grocery store after trying three bags over past week have never been sickeningly disappointed thought maybe first stale tried with same results doesnt even taste like popcorn alone cheese tried light sprinkle salt that made worse tasted nastybr from only buying generic popcorn from kroger marsh aldis adding seasoning
2,2,9.0,0.9750,"product, purchase, taste, slim, coffee, skin, fresh, flavor, smell, year",there many artificial ingredients this made read smell bourbon which just above salt then have better nose than hound want sodium benzoate yellow yellow blue partially hydrogenated soybean diglycerides artifical flavor sodium propionate this your fruitcake wish could send mine flavor very disappointing what expect with those chemical they need nuns make cake think monestary part just ploy
3,3,2.0,0.9864,"flavor, order, natural, coffee, taste, ingredient, amazon, think, look, jerky",well ordered sampler pack flavored coffee were grove square cappuccino mixes drink flavored coffee because dont partially hydrogenated vegetable contain coconut soybean sunflower palm andor canola sugar corn syrup solids instant coffee sodium caseinate salt natural artificial flavors cocoa processed with alkali dipotassium phosphate cornstarch mono diglycerides silicon dioxide lecithin artificial color sucralose body willinglybr cappuccino coffeebr that being said left with hazelnuts decaf didnt order decaf sampler caramel vanilla southern pecanbr happybr onnalee
4,4,2.0,0.9727,"flavor, order, natural, coffee, taste, ingredient, amazon, think, look, jerky",used this cereal when daughter baby years great ordered case recently month spoiled amazon great refunded money disappointed with happybellies there obviously some major quality control issues that need addressed will ordering baby cereal from different company just cant bring myself trust happybellies anymore
5,5,4.0,0.9400,"like, flavor, taste, sell, corn, zico, orange, think, water, item",took sips these nearly gagged think tastes spicy like nutmeg cant stand taste smell nutmeg cant drink this just cents
6,6,3.0,0.9591,"taste, like, chocolate, smell, food, come, price, coffee, tin, love",only this like taste chinese actually like chinese cheaper than this overpriced what does taste like american that what seeking choose another brand flowering effect that dramatic pretty taste llike full brewed only each once
7,7,4.0,0.9500,"like, flavor, taste, sell, corn, zico, orange, think, water, item",bought this same coffee walmart nowhere that instant actual wasnt packaged like normal after done there nothing left grounds anything didnt have built filter like most cups good coffee regardless
8,8,6.0,0.8329,"product, know, taste, buy, candy, try, store, tell, time, popcorn",purchased this product solely description from website stated that delicious many customers report they love spoon when read that thought great sold recieved this product mixed teaspoons with some skim milk drank damn near threw took choke down then even tried mixing teaspoons with bananas strawberries quart strawberry yogurt still undrinkable threw away cause want spend dollars other stuff just make teaspoons taste better dont this stuffbr
9,9,8.0,0.9816,"food, like, potato, weight, package, time, try, control, taste, flavor",close your eyes really hard might able pick feint hint candy corn flavor these oreos otherwise they just taste like overly sweet frosting

In [30]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,0.0,0.9870,"ball, container, think, treat, chew, try, product, away, good, liquid",decided this treat with trying teach different responses thought that treat that calories diet would good ideabr introduced treat ignored beginning shape container fashioned rollon deodorants moved ball around some liquid reintroduced liked flavor frustrated with ball gets stuck tried times ended yanking container from hand running away with itbr very fast when caught with managed ball container lucky swallow ball think could have killed picked ball threw away could container from busy using chew toybr will buying this product hard think that should chewing plastic contents were delicious they were chicken basedbr recommend this product
1,1.0,0.9780,"like, flavor, package, think, sugar, free, true, calorie, vendor, chew",couldnt begin review coconut arrived packaged such that every bags were fused together heat true that allowed return equally true that credited sale vendor must stupid company based arizona they didnt think that would melt brings into question business sense perfume even name indicates that therei would never order anything from them again contacted president email weeks hasnt even deigned answer that does speak well vendor represents amazoncom does
2,2.0,0.9866,"flavor, order, natural, coffee, taste, ingredient, amazon, think, look, jerky",disappointed with taste these having dried blueberries before they were sweet sickly unnatural sweet unlike blueberries previouslybr when looked ingredients clear whybr wild blueberries goodbr corn syrup theres cause ughbr sugar maybe okay corn syrup gone dont need much sugar with dried fruitbr natural flavor uhhh natural addedbr citric acidbr sunflower oilbr potassium sorbatebr ingredients could forgive them flavor right horrible donating rest food bank hoping someone likes refinedhypersweet tastes gets some enjoyment them going different supplier dried fruit needs enjoyed cherries past after this afraid anything else from this company
3,3.0,0.9830,"taste, like, chocolate, smell, food, come, price, coffee, tin, love",husband hamilton beach coffee makers that uses senseo pods when this coffee came pods decided since havent found other competitors senseo unfortunately this just when opened pouch smelled bitter when brewed smelled tasted burnt wasnt weak some other reviewers have stated tasted poured rest thecup down drainbr obviously food item matter individual taste however this smelled noncoffee drinker loves smell brewing coffee even flavored creamer couldnt make palatable husband price reasonable just cant recommend
4,4.0,0.9804,"like, flavor, taste, sell, corn, zico, orange, think, water, item",these huge disappointment compared pictures first there were pink monsters speak second while each blue orange green they specific each design monster lizard looking ones were orange blue that doesnt lead greatest disappointment they monochromatic subtle color shading white teeth just solid fluorescent color with some black eyes mouth cute attention detail complete waste money have never written review item amazoncom before have never been this severely disappointed please


In [31]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics.head(1000)

,Dominant_Topic,Topic_Keywords,Num_Documents,Perc_Documents
0.0,2.0,"flavor, order, natural, coffee, taste, ingredient, amazon, think, look, jerky",3.0,0.03
1.0,6.0,"product, know, taste, buy, candy, try, store, tell, time, popcorn",4.0,0.04
2.0,9.0,"product, purchase, taste, slim, coffee, skin, fresh, flavor, smell, year",14.0,0.14
3.0,2.0,"flavor, order, natural, coffee, taste, ingredient, amazon, think, look, jerky",7.0,0.07
4.0,2.0,"flavor, order, natural, coffee, taste, ingredient, amazon, think, look, jerky",12.0,0.12
...,...,...,...,...
995.0,NaN,NaN,NaN,NaN
996.0,NaN,NaN,NaN,NaN
997.0,NaN,NaN,NaN,NaN
998.0,NaN,NaN,NaN,NaN
